## Testing notebook playground for extracting data

In [2]:
import pandas as pd
import mysql.connector as msc
import json
import tweepy
import csv
import re
import string
import requests

In [3]:
# read config file into JSON object
configFile = open('config.json')
config = json.load(configFile)

# make connection to the GCloud database
db_config = config['database']
connection = msc.connect(
    host=db_config['host'], 
    port=db_config['port'],
    user=db_config['user'],
    password=db_config['password'],
    database=db_config['database']
)

# authorize Twitter app with Tweepy OAuthHandler
twit_config = config['twitter']
auth = tweepy.OAuthHandler(twit_config['api_key'], twit_config['api_key_secret'])
auth.set_access_token(twit_config['access_token'], twit_config['access_token_secret'])
twitapi = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
# extract Tweets from Malaysian MOH about Malaysian COVID-19 case data
# write to pandas database
tweet_collection = twitapi.search("KKMPutrajaya")
print(len(tweet_collection))

15


In [7]:
# Get user id of @KKMPutrajaya
url = "https://api.twitter.com"
headers = {"Authorization": "Bearer {}".format(twit_config['bearer_token'])}
response = requests.request("GET", url + "/2/users/by/username/KKMPutrajaya", headers=headers)
user_id = response.json()['data']['id']
user_id

'531041640'

In [29]:
# initialize pandas data set
tweets = []

In [30]:
# get posts from Malaysian MOH (@KKMPutrajaya)
search_url = (url + "/2/users/{}/tweets".format(user_id))
response = requests.request("GET", search_url, headers=headers)
data = response.json()['data']

In [31]:
# cycle and add to data
for tweet in data:
    if "Status Terkini #COVID19" in tweet['text']:
        tweets.append(tweet)

In [32]:
tweets

[{'id': '1406914442466447363',
  'text': 'Status Terkini #COVID19, 21 Jun 2021 \nKes sembuh= 5,439\nJumlah kes sembuh= 633,624\nKes baharu positif= 4,611 (21 import) \nJumlah positif= 701,019\nKes kematian= 69\nJumlah kes kematian= 4,477\nKes dirawat di ICU= 880\nBantuan alat pernafasan= 452 https://t.co/SKuAbcwQyK'}]